# StyleGAN2-ADA

## Установка

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf

# Download the code
!git clone https://github.com/NVlabs/stylegan2-ada.git
%cd stylegan2-ada
!nvcc test_nvcc.cu -o test_nvcc -run

print('Tensorflow version: {}'.format(tf.__version__) )
!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

In [ ]:
import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
from io import BytesIO
import IPython.display
import numpy as np
from math import ceil
from PIL import Image, ImageDraw
import imageio
import os
import pickle
from google.colab import files

### Добавляем функции для генерации изображений

In [ ]:
# Useful utility functions...

# Generates a list of images, based on a list of latent vectors (Z), and a list (or a single constant) of truncation_psi's.
def generate_images_in_w_space(dlatents, truncation_psi):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    Gs_kwargs.truncation_psi = truncation_psi
    dlatent_avg = Gs.get_var('dlatent_avg') # [component]

    imgs = []
    for row, dlatent in log_progress(enumerate(dlatents), name = "Generating images"):
        #row_dlatents = (dlatent[np.newaxis] - dlatent_avg) * np.reshape(truncation_psi, [-1, 1, 1]) + dlatent_avg
        dl = (dlatent-dlatent_avg)*truncation_psi   + dlatent_avg
        row_images = Gs.components.synthesis.run(dlatent,  **Gs_kwargs)
        imgs.append(PIL.Image.fromarray(row_images[0], 'RGB'))
    return imgs       

def generate_images(zs, truncation_psi):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if not isinstance(truncation_psi, list):
        truncation_psi = [truncation_psi] * len(zs)
        
    imgs = []
    for z_idx, z in log_progress(enumerate(zs), size = len(zs), name = "Generating images"):
        Gs_kwargs.truncation_psi = truncation_psi[z_idx]
        noise_rnd = np.random.RandomState(1) # fix noise
        tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(z, None, **Gs_kwargs) # [minibatch, height, width, channel]
        imgs.append(PIL.Image.fromarray(images[0], 'RGB'))
    return imgs

def generate_zs_from_seeds(seeds):
    zs = []
    for seed_idx, seed in enumerate(seeds):
        rnd = np.random.RandomState(seed)
        z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        zs.append(z)
    return zs

# Generates a list of images, based on a list of seed for latent vectors (Z), and a list (or a single constant) of truncation_psi's.
def generate_images_from_seeds(seeds, truncation_psi):
    return generate_images(generate_zs_from_seeds(seeds), truncation_psi)

def saveImgs(imgs, location):
  for idx, img in log_progress(enumerate(imgs), size = len(imgs), name="Saving images"):
    file = location+ str(idx) + ".png"
    img.save(file)

def imshow(a, format='png', jpeg_fallback=True):
  a = np.asarray(a, dtype=np.uint8)
  str_file = BytesIO()
  PIL.Image.fromarray(a).save(str_file, format)
  im_data = str_file.getvalue()
  try:
    disp = IPython.display.display(IPython.display.Image(im_data))
  except IOError:
    if jpeg_fallback and format != 'jpeg':
      print ('Warning: image was too large to display in format "{}"; '
             'trying jpeg instead.').format(format)
      return imshow(a, format='jpeg')
    else:
      raise
  return disp

def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

        
def clamp(x, minimum, maximum):
    return max(minimum, min(x, maximum))
    
def drawLatent(image,latents,x,y,x2,y2, color=(255,0,0,100)):
  buffer = PIL.Image.new('RGBA', image.size, (0,0,0,0))
   
  draw = ImageDraw.Draw(buffer)
  cy = (y+y2)/2
  draw.rectangle([x,y,x2,y2],fill=(255,255,255,180), outline=(0,0,0,180))
  for i in range(len(latents)):
    mx = x + (x2-x)*(float(i)/len(latents))
    h = (y2-y)*latents[i]*0.1
    h = clamp(h,cy-y2,y2-cy)
    draw.line((mx,cy,mx,cy+h),fill=color)
  return PIL.Image.alpha_composite(image,buffer)
             
  
def createImageGrid(images, scale=0.25, rows=1):
   w,h = images[0].size
   w = int(w*scale)
   h = int(h*scale)
   height = rows*h
   cols = ceil(len(images) / rows)
   width = cols*w
   canvas = PIL.Image.new('RGBA', (width,height), 'white')
   for i,img in enumerate(images):
     img = img.resize((w,h), PIL.Image.ANTIALIAS)
     canvas.paste(img, (w*(i % cols), h*(i // cols))) 
   return canvas

def convertZtoW(latent, truncation_psi=0.7, truncation_cutoff=9):
  dlatent = Gs.components.mapping.run(latent, None) # [seed, layer, component]
  dlatent_avg = Gs.get_var('dlatent_avg') # [component]
  for i in range(truncation_cutoff):
    dlatent[0][i] = (dlatent[0][i]-dlatent_avg)*truncation_psi + dlatent_avg
    
  return dlatent

def interpolate(zs, steps):
   out = []
   for i in range(len(zs)-1):
    for index in range(steps):
     fraction = index/float(steps) 
     out.append(zs[i+1]*fraction + zs[i]*(1-fraction))
   return out

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )



## Подключение Гугл.Диска

Подключаем Диск, нужно будет скопировать и вставить код

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Создаем на диске папку, куда будем сохранять наши модели и результаты. 

Эту ячейку нужно запустить только один раз. Когда вы решите снова воспользоваться этим ноутбуком, ячейку можно уже будет не запускать, т.к. папки с диска никуда не исчезнут.

In [ ]:
!mkdir /content/drive/My\ Drive/stylegan
!mkdir /content/drive/My\ Drive/stylegan/models
!mkdir /content/drive/My\ Drive/stylegan/results

## Скачиваем натренированные модели

| Название | Разрешение | Эпохи | Ссылка |
|---|---|---|---|
| **Лица** | 1024x1024 | 15000 | http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl |
| **Машины** | 512x384 | 15000 | http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-car-config-f.pkl |
| **Лошади** | 256x256 | 15000 | http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-horse-config-f.pkl |
| **Коты** | 256x256 | 15000 | https://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-cat-config-f.pkl |
| **Церкви** | 256x256 | 15000 | https://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-church-config-f.pkl |
| **ImageNet** | 512x512 | 533504 | https://battle.shawwn.com/sdc/stylegan2-imagenet-512/model.ckpt-533504.pkl |



Эти датасеты надо вручную скачать и сложить к себе на Гугл.Диск (позже переложу куда-то):

<strike> **Аниме** 512x512 : https://mega.nz/#!PeIi2ayb!xoRtjTXyXuvgDxSsSMn-cOh-Zux9493zqdxwVMaAzp4

**Микроорганизмы** 1024x1024 : https://mega.nz/#!PbgzWTZT!JbVpqgMU7AOg-sQUoG1BDepuwKtgAsLgjd4YwlTXlpc

**Абстрактное искусство** 1024x1024 : https://drive.google.com/uc?id=1YzZemZAp7BVW701_BZ7uabJWJJaS2g7v </strike> 



In [ ]:
%cd /content/drive/My\ Drive/stylegan/models/
!wget http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl

## Загружаем модели в нейросеть

In [ ]:
%cd /content/stylegan2-ada

In [ ]:
dnnlib.tflib.init_tf()

# Здесь указываем какую модель мы хотим использовать для генерации!
network_pkl = '/content/drive/My Drive/stylegan/models/stylegan2-ffhq-config-f.pkl'
 
print('Загружаем модель "%s"...' % network_pkl)
with dnnlib.util.open_url(network_pkl) as fp:
    _G, _D, Gs = pickle.load(fp)
noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

## Генерация

### Одно изображение

In [ ]:
imshow(generate_images_from_seeds(np.random.randint(4294967295, size=1), truncation_psi=0.5)[0])

### Сетка изображений

```num_images``` - количество изображений

```trunc_psi``` - разнообразие изображений (0 - усреднённое значение, 1 максимальное разнообразие)

```scale``` - масштаб изображений

```rows``` - количество строк в сетке

In [ ]:
num_images = 9
trunc_psi= 0.7

scale = 0.7
rows = 3

seeds = np.random.randint(4294967295, size=num_images)
imshow(createImageGrid(generate_images_from_seeds(seeds, trunc_psi), scale , rows))

### Несколько изображений

Генерируем несколько изображений и сохраняем их в папку на Гугл Диске

```num_images``` - количество изображений

```trunc_psi``` - разнообразие изображений (0 - усреднённое значение, 1 максимальное разнообразие)

```location``` - папка с результатами

In [ ]:
# создаем папку для результата
!mkdir /content/drive/My\ Drive/stylegan/results/01

In [ ]:
num_images = 20
trunc_psi= 0.7
location = '/content/drive/My Drive/stylegan/results/01/' # важно чтоб в конце стоял /

seeds = np.random.randint(4294967295, size=num_images)

imgs = generate_images_from_seeds(seeds, trunc_psi)

saveImgs(imgs, location)

### Интерполяция между двумя изображениями

```number_of_steps``` - количество шагов интерполяции

```trunc_psi``` - разнообразие изображений (0 - усреднённое значение, 1 максимальное разнообразие)

```scale``` - масштаб изображений

```rows``` - количество строк в сетке

```seed1, seed2``` - "адреса" изображений (сейчас случайные числа, но можно задать и какие-то конкретные)

In [ ]:
number_of_steps = 10
trunc_psi = 0.5

scale = 0.4
rows = 1

seed1 = np.random.randint(4294967295, size=1)
seed2 = np.random.randint(4294967295, size=1)



zs = generate_zs_from_seeds([seed1, seed2])
print(seed1, seed2)


imgs = generate_images(interpolate(zs,number_of_steps), trunc_psi)

imshow(createImageGrid(imgs, scale , rows))

### Видео интерполяция

```num_images``` - количество изображений

```number_of_steps``` - количество шагов интерполяции

```trunc_psi``` - разнообразие изображений (0 - усреднённое значение, 1 максимальное разнообразие)

```movie_name``` - путь, куда мы сохраняем наше видео

In [ ]:
# создаем папку для результата
!mkdir /content/drive/My\ Drive/stylegan/results/02

In [ ]:
num_images = 20
number_of_steps = 10
trunc_psi = 0.5
movie_name = "/content/drive/My Drive/stylegan/results/02/interpolate.mp4"


seeds = list(np.random.randint(4294967295, size=num_images))
seeds = seeds + [seeds[0]]
zs = generate_zs_from_seeds(seeds)


imgs = generate_images(interpolate(zs,number_of_steps), trunc_psi)

with imageio.get_writer(movie_name, mode='I') as writer:
    for image in log_progress(list(imgs), name = "Creating animation"):
        writer.append_data(np.array(image))